In [1]:
import pandas as pd
import numpy as np

housing = pd.read_csv('manhattan_rentals_apr27.csv')
neighborhoods = [x.strip() for x in list(housing['Neighborhood'].value_counts().index)]
neighborhood_options = [{'label': x, 'value': x} for x in neighborhoods]
full_df = pd.read_csv('scored_housing/full_housing.csv')
full_df['Monthly Rental Price'] = full_df['Monthly Rental Price'].astype(float)

import chart_studio.plotly as py
from plotly.offline import plot
import plotly.graph_objs as go
import plotly
import chart_studio as cs
import pandas as pd



## Creating the User Intake Survey on Dash

In [ ]:
dislikes = []

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc



# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css', dbc.themes.JOURNAL]
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

cs.tools.set_credentials_file(username='apartmentranker', api_key='pk.eyJ1IjoiYXBhcnRtZW50cmFua2VyIiwiYSI6ImNrOWk0MGphdTAzY3QzZXA1NXZkNzB3MjIifQ.633TMyX6NjcaiooIn6204g')
mapbox_access_token = 'pk.eyJ1IjoiYXBhcnRtZW50cmFua2VyIiwiYSI6ImNrOWk0MGphdTAzY3QzZXA1NXZkNzB3MjIifQ.633TMyX6NjcaiooIn6204g'

first_recommendation = 0


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


app.config['suppress_callback_exceptions']=True

navbar = html.Div(
    children=[
#         html.Div(["ApartmentRankers"]),
        html.Div([
        dcc.Location(id='url', refresh=False),

    dcc.Link('Home', href='/', style = {'text-decoration': 'none !important'}),
#     html.Br(),
#         html.Pre(),
#             html.A()
#         dcc.Markdown(children="""   """),

            html.A("     "),
            dcc.Link('About Us', href='/about-us', style = {'text-decoration': 'none !important'})
    ],style={
            'float': 'right',
            'display': 'inline-block',
            'color': '#f2f2f2',
            'text-align': 'right',
          'padding': '14px',
          'text-decoration': 'none ! important',
            'font-size':'15px',
            'display': 'inline'
            
        })], style={
         'text-decoration': 'none ! important',
        'textAlign':'right',
        'background-color': '#333',
      'top': '0',
      'width': '100%',
        'height' : '50px'
           }
)

# add callback for toggling the collapse on small screens



footer = html.Footer([
    html.Div(children=["Made with ❤️ in Berkeley, CA."], style={
        
        'color': 'white',
        'font-size': '16px'
        
    })],style={
            'textAlign':'left',
            'background-color': '#333',
            'overflow': 'hidden',
            'position': 'relative',
            'bottom': '0',
            'width': '100%',
            'height': '25px',
            'padding': '10px'
           })

fig = go.Figure()
app.layout = html.Div(children=[
    
    navbar,
    
    
    html.Div(id='display'),
#     dcc.Graph(id = 'plot', figure = fig),
#     dcc.Store(id='figure_html'),
#     dcc.Graph(id='live-update-graph'),
#     dcc.Interval(
#             id='interval-component',
#             interval=1*1000, # in milliseconds
#             n_intervals=0
#         ),
    
    
    
    footer


    ])


@app.callback(dash.dependencies.Output('display', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display(pathname):
    if pathname == '/about-us':

        return html.Iframe(srcDoc = str(open('about.html', 'r').read()), width = 1500, height = 1050)
    
    
    if pathname == "/":
        return html.Div([html.Div(children=[
            
              html.H1(children=["Welcome to ApartmentRanker v.2!"],
           style={
               'textAlign':'center'
           }),
            
        html.H3(children="Tell us about yourself!",
               style={
                   'textAlign':'center'
               }),
        
        html.Div([
            dcc.Markdown(children=
                        """
                        ApartmentRanker works best if you answer each question honestly as we use this data to match suitable apartments with you. Simply fill out the required questions and see apartments around you today!
                        """)
        ]),
        
        html.Div(children=[
            
            html.H5(children="First, let's talk requirements.",
                    style={'textAlign':'center'}),
            
            html.Div(children=[
                html.Label('Number of bedrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bedrooms-state',
                    options=[
                        {'label': '0 (studio)','value':0.0},
                        {'label': '1 bedroom','value':1.0},
                        {'label': '2 bedrooms','value':2.0},
                        {'label': '3 bedrooms','value':3.0},
                        {'label': '4 bedrooms','value':4.0},
                        {'label': '5 bedrooms','value':5.0}
                    ]),

                html.Label('Number of bathrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bathrooms-state',
                    options=[
                        {'label': '1 bathroom','value':1.0},
                        {'label': '2 bathrooms','value':2.0},
                        {'label': '3 bathrooms','value':3.0},
                        {'label': '4 bathrooms','value':4.0},
                        {'label': '5 bathrooms','value':5.0}
                    ]),

                ],style={'columnCount':'2'}),
        
            html.Label('Maximum monthly rental price',style={"font-weight":"bold"}),
            dcc.Slider(
                id='price_max-state',
                min=1000,
                max=10000,
                marks={i: '${}'.format(i) for i in range (1000,11000,1000)}
            ),
            
            html.Label('Preferred neighborhoods',style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='neighborhood-state',
                options=neighborhood_options,
                multi=False)
        
        ]),
        
        html.Div(children=[
            
            html.H5(children="One last thing, tell us about your priorities!",
                    style={'textAlign':'center'}),
            
            html.Label("1st priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority1-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("2nd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority2-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),    
                 
            html.Label("3rd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority3-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),   
                 
            html.Label("4th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority4-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("5th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority5-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of srestaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),  
                 
            html.Label("6th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority6-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),                 
            ]),
        
        html.Button(id='submit-button-state', children='Submit'), 
            html.Div(id='initial_rec',  
            style=
            {'textAlign':'center'
            })
            
#             , html.Div(id= 'figure_html', style=
#             {'textAlign':'center'
#             })
            
#             ,dcc.Graph(id = 'plot', figure = fig)
        
        ],
            style=
            {"margin":"0 auto",
             'max-width':"650px",
             'textAlign':'center'
            })]) 
    


@app.callback(
    Output('initial_rec', 'children'),
    [Input('submit-button-state','n_clicks')],
    [State('bedrooms-state','value'),
    State('bathrooms-state','value'), 
    State('price_max-state','value'),
    State('neighborhood-state','value'),
    State('priority1-state','value'),
    State('priority2-state','value'),
    State('priority3-state','value'),
    State('priority4-state','value'),
    State('priority5-state','value'),
    State('priority6-state','value')])
def initial_rec(n_clicks,beds,bath,price,neighbor,p1,p2,p3,p4,p5,p6):
    global first_recommendation
    #preferences is a list with the ranked features of
    # grocers_normalized
    # nightlife_normalized
    # noise_normalized
    # restaurants_normalized
    # subways_normalized
    # trees_normalized
    

    if n_clicks < 1:
        return ""

    
    counter=1

    tree_pref = 0
    subway_pref = 0
    grocer_pref = 0
    noise_pref = 0
    restaurants_pref = 0
    nightlife_pref = 0

    p1="nightlife"
    p2="food"
    p3="noise"
    p4="grocer"
    p5="subway"
    p6="trees"

    for p in [p1,p2,p3,p4,p5,p6]:
        if p=="trees":
            tree_pref=counter
            counter+=1
            continue
        if p=="subway":
            subway_pref=counter
            counter+=1
            continue
        if p=="grocer":
            grocer_pref=counter
            counter+=1
            continue
        if p=="noise":
            noise_pref=counter
            counter+=1
            continue
        if p=="food":
            restaurants_pref=counter
            counter+=1
            continue
        if p=="nightlife":
            nightlife_pref=counter
            counter+=1
            continue
            
    preferences = np.array([grocer_pref, nightlife_pref, noise_pref, restaurants_pref, subway_pref, tree_pref])
    
    new_df = full_df[(full_df['Bedrooms'] == beds) & (full_df['Bathrooms'] == bath) & (full_df['Monthly Rental Price'] <= price) & (full_df['Neighborhood'] == neighbor)]
    new_df["User Score"] = 0.0
    
    scores = []
    for i in range(new_df.shape[0]):
        score = preferences * np.array(new_df.iloc[i, 19:25] + 10)
        scores.append(sum(score))
    new_df['User Score'] = scores
    new_df = new_df.sort_values(by = ['User Score'], ascending = False)
    
    counter_1 = 0
#     print(new_df.head())
    try:
        while True:
            potential = new_df.iloc[counter_1, :]
            if potential not in dislikes:
                break
            else:
                counter_1 = counter_1 + 1
        result = potential
        first_recommendation = potential
        rec = "We recommend: " + result['Location'] + " in " + result['Neighborhood'] + ". This listing is $" + str(int(result['Monthly Rental Price'])) + " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms."
        
        fig = go.Figure(go.Scattermapbox(
        lat=pd.Series(first_recommendation["Latitude"]),
        lon=pd.Series(first_recommendation["Longitude"]),
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=first_recommendation["Location"],
        )
    )

        fig.update_layout(
        hovermode="closest",
        mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=first_recommendation["Latitude"],
            lon=first_recommendation["Longitude"]
        ),
        pitch=0,
        zoom=15
        )
    )

        trace_1 = go.Scattermapbox(
        lat=pd.Series(first_recommendation["Latitude"]),
        lon=pd.Series(first_recommendation["Longitude"]),
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=first_recommendation["Location"],
    )


        layout = go.Layout(
        hovermode="closest",
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=first_recommendation["Latitude"],
                lon=first_recommendation["Longitude"]
            ),
            pitch=0,
            zoom=15
        )
    )


        fig = go.Figure(data = [trace_1], layout = layout)

        htmlPlot = html.Div([dcc.Graph(id = 'plot', figure = fig)])
        return (rec, htmlPlot)
    except:
        return "There is currently no listing with the given criteria."
    
    
    
# @app.callback(dash.dependencies.Output('figure_html', 'children'),[Input('submit-button-state','n_clicks')]) 
# # @app.callback(Output('live-update-graph', 'figure'),
# #               [Input('interval-component', 'n_intervals')])
# def figure_html(n_clicks):
#     global fig
#     print(n_clicks)
# #     if n_clicks < 1:
# #         return ""
    
    
#     print(first_recommendation)
# #     return "test"
#     try:

#         fig = go.Figure(go.Scattermapbox(
#         lat=pd.Series(first_recommendation["Latitude"]),
#         lon=pd.Series(first_recommendation["Longitude"]),
#         mode="markers",
#         marker=go.scattermapbox.Marker(
#             size=14
#         ),
#         text=first_recommendation["Location"],
#         )
#     )

#         fig.update_layout(
#         hovermode="closest",
#         mapbox=dict(
#         accesstoken=mapbox_access_token,
#         bearing=0,
#         center=go.layout.mapbox.Center(
#             lat=first_recommendation["Latitude"],
#             lon=first_recommendation["Longitude"]
#         ),
#         pitch=0,
#         zoom=15
#         )
#     )

#         trace_1 = go.Scattermapbox(
#         lat=pd.Series(first_recommendation["Latitude"]),
#         lon=pd.Series(first_recommendation["Longitude"]),
#         mode="markers",
#         marker=go.scattermapbox.Marker(
#             size=14
#         ),
#         text=first_recommendation["Location"],
#     )


#         layout = go.Layout(
#         hovermode="closest",
#         mapbox=dict(
#             accesstoken=mapbox_access_token,
#             bearing=0,
#             center=go.layout.mapbox.Center(
#                 lat=first_recommendation["Latitude"],
#                 lon=first_recommendation["Longitude"]
#             ),
#             pitch=0,
#             zoom=15
#         )
#     )


#         fig = go.Figure(data = [trace_1], layout = layout)

#         htmlPlot = html.Div([dcc.Graph(id = 'plot', figure = fig)])
#         return htmlPlot
#     except:
#         return ""
    
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2020 19:19:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:19:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:19:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:19:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 19:19:02] "POST /_dash-update-component HTTP/1.1" 200 -
[2020-05-03 19:19:03,051] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Anderson\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Anderson\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Anderson\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Anders